In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F


#preparing the data : Given a file, the function prepares 2 arrays
"""
feature_set : each element is an array of 50 readings
one_hot_labels : each element is a label corrisponding to the readings with the same index in feature_set
"""
def getSamplesFromFile(fileName="mHealth_ECGProcessed_subject3.log"):
    df = pd.read_csv(fileName, header=None, delim_whitespace=True)

    one_hot_labels  = np.array([])
    feature_set= np.array([])
    for i in range(int(df.shape[0]/50)):
        if df[2][i*50:i*50+50].mean() in range(1,13):
            feature_set=np.append(feature_set,[df[1][i*50:i*50+50]])
            one_hot_labels=np.append(one_hot_labels,[[0]*(df[2][i*50]-1)+[1]+[0]*(12-df[2][i*50])])
    #     else:
    #         print("tt: ",i)
    feature_set=feature_set.reshape(int(feature_set.shape[0]/50),50)
    one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set, one_hot_labels

In [ ]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 25
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(50, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, 12)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 50)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = self.fc2(x)        
        return x

# initialize the NN
model = Net()

In [ ]:
feature_set,one_hot_labels=getSamplesFromFile()

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
feature_set=torch.from_numpy(feature_set).float()
one_hot_labels=torch.from_numpy(one_hot_labels).long()
# number of epochs to train the model
n_epochs = 50000000

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
  
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(feature_set)
      # calculate the loss
    loss = criterion(output, torch.max(one_hot_labels, 1)[1])    
    
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    
   
    print('Epoch: {} \tTraining Loss: {:.6f} '.format(
        epoch+1, 
        loss.item()
        ))